<a href="https://colab.research.google.com/github/JaiswalFelipe/Learning-Python/blob/main/Practical_NumPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Loan Data Practical Example with NumPy**
- What working as a data analyst in a data science team looks like
  - Gathering --> Cleaning --> Preprocessing + Notes about the changes
<br></br>
- Team task:
  - Create a credit risk model, which estimates the *probability of default* for every personal account
<br></br>
- STEP-BY-STEP:
  1. Clean the dataset
  2. Preprocess the dataset
  3. Prepare the dataset for further analysis 
<br></br>
- Additional steps to remember:
  - Step 1: Convert USD --> EU
  - Step 2: Every categorical variable must be quantified
    - Change any text columns into **numbers**
<br></br>
- OBS: When measuring creditworthiness
  - Be extremely risk-averse and distrustful
  - Missing info suggests foul play
  - If the info isn't available, assume the worst


In [124]:
import numpy as np

### **Set Up**

##### The Dataset

In [125]:
# For easier viewing, avoid scientific notation (suppress = True), and prints 2 digits after the decimal point (precision = 2)
# These arguments will only affect how we "see" their values on screen, not on calcus
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

In [126]:
# Has NANS so np.genfromtxt()
raw_data_np = np.genfromtxt("/content/drive/MyDrive/Colab Notebooks/365 Data Science/Programming/Numpy/Files for the Practical Example/loan-data.csv",
                            delimiter = ";",
                            encoding = "ISO-8859-1",    # Solve encoding/UFT error
                            skip_header = 1,            # Skips first row (the NANs)
                            autostrip = True)           # Removes excess whitespaces
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

In [127]:
raw_data_np.shape

(10000, 14)

##### Checking for Incomplete Data

In [128]:
np.isnan(raw_data_np).sum()

88005

In [129]:
# A filler for all the missing entries of the dataset
temporary_fill = np.nanmax(raw_data_np) + 1

# Hold the means for every column
temporary_mean =  np.nanmean(raw_data_np, axis = 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [130]:
# Examine the warning: Mean of empty slice
temporary_mean

# Here, any column with a mean of "nan" contains no numbers

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [131]:
# Temporary Statistics: nanmin and nanmax
# Extract minimum, mean and maximum values for each numeric column
temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
                            temporary_mean,
                            np.nanmax(raw_data_np, axis = 0)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: All-NaN slice encountered
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: All-NaN slice encountered
  """


In [132]:
# Temporary mean is the "2nd" stats of the array
# min
# mean
# max
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

##### Splitting the Dataset

In [133]:
# Splitting the Columns
# Remember, the temporary_mean contains strings
# Extracting columns with "strings"
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [134]:
# Extracting columns which contains "numeric values"
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numeric

array([ 0,  2,  4,  6,  7, 13])

In [135]:
# Re-importing the Dataset: The columns with strings
loan_data_str = np.genfromtxt("/content/drive/MyDrive/Colab Notebooks/365 Data Science/Programming/Numpy/Files for the Practical Example/loan-data.csv",
                             delimiter = ";",
                             encoding = "ISO-8859-1",    # Solve encoding/UFT error
                             skip_header = 1,            # Skips first row (the NANs)
                             autostrip = True,           # Removes excess whitespaces
                             usecols = columns_strings,  # The columns with strings
                             dtype = np.str)             # Set dtype as str 

loan_data_str

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [136]:
# Re-importing the Dataset: The columns with numeric values
loan_data_num = np.genfromtxt("/content/drive/MyDrive/Colab Notebooks/365 Data Science/Programming/Numpy/Files for the Practical Example/loan-data.csv",
                             delimiter = ";",
                             encoding = "ISO-8859-1",           # Solve encoding/UFT error
                             skip_header = 1,                   # Skips first row (the NANs)
                             autostrip = True,                  # Removes excess whitespaces
                             usecols = columns_numeric,         # The columns with numeric values
                             filling_values = temporary_fill)   # Fill missing values  

loan_data_num

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [137]:
# Then, the Names of the Columns
header_full = np.genfromtxt("/content/drive/MyDrive/Colab Notebooks/365 Data Science/Programming/Numpy/Files for the Practical Example/loan-data.csv",
                             delimiter = ";",
                             encoding = "ISO-8859-1",           # Solve encoding/UFT error
                             skip_footer = raw_data_np.shape[0],# Set it to the Number of rows of raw_data_np(This ignores all rows after the header)
                             autostrip = True,                  # Removes excess whitespaces 
                             dtype = np.str)                    # Set dtype to str 

header_full          

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [138]:
# Separate the headers for strings and numeric vals
header_str, header_num = header_full[columns_strings], header_full[columns_numeric]

display(header_str)
display(header_num)

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

#####  Creating Checkpoints
- Places throughout our code where we store a copy of our dataset (or only parts of it)
- To avoid losing a lot of progress
- Extremely reliable practice when we need to clean or preprocess many parts of a dataset

In [139]:
# This function will take the header and data arguments we provide
def checkpoint(file_name, checkpoint_header, checkpoint_data):  
  np.savez(file_name, header = checkpoint_header, data = checkpoint_data) # And create an "npz" file we have assigned
  checkpoint_variable = np.load(file_name + ".npz")                       # Load
  return(checkpoint_variable)                                             # Return the loaded variable as an output of the function

In [140]:
checkpoint_test = checkpoint("checkpoint-test", header_str, loan_data_str)

#checkpoint_test['header']    # Uncomment to see output
checkpoint_test['data']   

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [141]:
# To make sure they are identical, call np.array_equal()
np.array_equal(checkpoint_test['data'], loan_data_str)

# TRUE == The checkpoint function works as intended

True

### **Manipulating Text Data**

In [142]:
header_str

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

##### Issue Date (issue_d)

In [143]:
header_str[0] = 'issue_date'

In [144]:
# Unique values of the first column 'issue_date'
np.unique(loan_data_str[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [145]:
# Notice: month-15 means that all the data is from 2015 so we can remove "-15" without losing any data
# Overwrite
loan_data_str[:,0] = np.chararray.strip(loan_data_str[:,0], "-15")

In [146]:
# After strip
np.unique(loan_data_str[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

In [147]:
# In analysis, represent MONTH VALUES as INT == Less memory
# Make an array with the proper order of the unique values
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Missing values ('') will be replaced with 0, Jan to 1, until Dec is 12 

In [148]:
# Using a for loop to assign the numerical equivalent to the unique values
for i in range(13):
# Function checks whether the value from the column equals a specific value from the "month" array
  loan_data_str[:,0] = np.where(loan_data_str[:,0] == months[i],   
                                i,                               # If so, assign corresponding numerical value 
                                loan_data_str[:,0])              # If not, remain unchanged

In [149]:
# Verify that there is no ('') whitespace in the array
np.unique(loan_data_str[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

##### Loan Status

In [150]:
# Check the unique values
np.unique(loan_data_str[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [151]:
# Remember the team task: 
# Regressions only care if the candidate is in a stable financial condition
# So split all possible values into (good vs bad)
# GOOD: "Fully Paid", "In Grace Period", "Issued", "Current", 'Late (16-30 days)' Clients pay when they get their salary at the end of the month 
# Bad: "Charged off", "Default", ''(No data), 'Late (31-120 days)' Late 3 months == Can't keep up
# 1 will represent GOOD, and 0 == Bad

status_bad = np.array(['','Charged Off', 'Default', 'Late (31-120 days)'])
# It is important to follow the order of how they appeared in the original array
# Otherwise, the "where" function won't properly assign the values to all the bad outcomes

In [152]:
# Combine "np.where()" and "np.isin()" functions to check if the loan status for a given row is included in "status_bad"
loan_data_str[:,1] = np.where(np.isin(loan_data_str[:,1], status_bad), 0,1)        # If so, set it to 0 and If not, 1

In [153]:
# Verify
np.unique(loan_data_str[:,1])

# Now, "loan status" column is transformed into a dummy variable

array(['0', '1'], dtype='<U69')

##### Term

In [154]:
np.unique(loan_data_str[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [155]:
header_str[2] = 'term_months'

In [156]:
# Remove the "months"
loan_data_str[:,2] = np.chararray.strip(loan_data_str[:,2], " months")
loan_data_str[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [157]:
# Assign 60 to each empty space
# Because if "NO DATA" == ASSUME THE WORST (60 months == 5 years is a long period) 
loan_data_str[:,2] = np.where(loan_data_str[:,2] == '', '60', 
                              loan_data_str[:,2]) # If not, remain unchanged

In [158]:
# Check contents of 'term'
np.unique(loan_data_str[:,2])

array(['36', '60'], dtype='<U69')

In [159]:
# If we only have 2 values, we can just USE 0 and 1 instead
# BUT ASK FIRST THE ONE THAT SCRAPED THE DATASET WHY THEY BELIEVE THAT 36 and 60 IS THE BETTER ALTERNATIVE
# In our example, we'll assume that there's no one to ask, and there is a significance to these values. So we won't change them.

##### Grade and Subgrade

In [160]:
# grade
np.unique(loan_data_str[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [161]:
# subgrade
np.unique(loan_data_str[:,4])

# For every elements in "grade", we find "5" different elements in "subgrade"
# So the information we can get from "grade", can also be obtained from "subgrade" therefore "grade" will be redundant
# However, that's not always the case in practice

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

In [162]:
# Filling subgrade

# For every element in "grade" excluding index 0 which is the empty space
for i in np.unique(loan_data_str[:,3])[1:]:
# If "subgrade" is empty AND has a corresponding value in "grade"
  loan_data_str[:,4] = np.where((loan_data_str[:,4] == '') & (loan_data_str[:,3] == i),
  i + '5',              # Set the corresponding value from "grade" and set 5 next to it (which is the worst subgrade of that grade) 
  loan_data_str[:,4])   # Otherwise remain unchanged

In [163]:
# Now check if there's still missing data in "subgrade"
np.unique(loan_data_str[:,4], return_counts = True)

# Which there is: meaning we neither have "subgrade", nor the "grade" (NO DATA)
# There are 9 instances of this case, shown by the 2nd array
# and 9 out of 10,000 (our original data) is relatively small
# So drop these values and continue with our analysis
# ANSWER FROM 365 HERE

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

In [164]:
# BUT there's a better way (for our credit risk analysis)
# The task requires you to assign status to every individual
# So instead of dropping, CREATE a whole new category LOWER THAN G5(the lowest)
loan_data_str[:,4] = np.where(loan_data_str[:,4] == '',
                              'H1',                         # If this is the case, assign H1
                              loan_data_str[:,4])

In [165]:
# To verify
np.unique(loan_data_str[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [166]:
# Now the information in "grade" is CARRIED by the information in "subgrade"
# So we "don't need" "grade" anymore
# So can be removed from the dataset

loan_data_str = np.delete(loan_data_str, 3, axis = 1) # 3 is the index, "axis = 1" means column
loan_data_str[:,3]

# Now index 3 will be "subgrade" instead of "grade"

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [167]:
# Now, we should also delete the "grade" in the header_str
header_str = np.delete(header_str, 3)

In [168]:
# To check previous cell
header_str[3]

'sub_grade'

In [169]:
# Converting "subgrade" to numbers

# Create a dictionary: A1 will be 1, A2 will be 2 (LEVEL OF TRUSTWORTHINESS)
keys = list(np.unique(loan_data_str[:,3]))                              # Consists of all the unique subgrades
values = list(range(1, np.unique(loan_data_str[:,3]).shape[0] + 1))     # A list of all the integers between 1 and 36 (One for each subgrade)
                                                                        # Without "+ 1" we will only have 35 values (for 36 subgrades)
dict_sub_grade = dict(zip(keys, values))

In [170]:
# Check the dictionary
print(dict_sub_grade)

{'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30, 'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35, 'H1': 36}


In [171]:
# Iterate through all the unique "subgrades" --> SUBSTITUTE each one with its associated numeric value
for i in np.unique(loan_data_str[:,3]):
  loan_data_str[:,3] = np.where(loan_data_str[:,3] == i,
                                dict_sub_grade[i],
                                loan_data_str[:,3])

In [172]:
# To check
np.unique(loan_data_str[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

##### Verification Status

In [173]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [174]:
# Check its values virst
# Then unique values
np.unique(loan_data_str[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [175]:
# Since we tend to be skeptical when the information is missing, we'll assume that ('') == not verified
# 'Source Verified' == loan applications which include investor backing
# '', 'Not Verified' == BAD --> 0
# 'Source Verified', 'Verified' == GOOD --> 1
loan_data_str[:,4] = np.where((loan_data_str[:,4] == '') | (loan_data_str[:,4] == 'Not Verified'), 0, 1)   # | means 'or'
# If condition is met, set to 0, if not == 1

In [176]:
# To check
np.unique(loan_data_str[:,4])

array(['0', '1'], dtype='<U69')

##### Url

In [177]:
loan_data_str[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [178]:
# The Urls are identical EXCEPT for the last part which is the "id"
# So we can strip the URL without losing info
loan_data_str[:,5] = np.chararray.strip(loan_data_str[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')
loan_data_str[:,5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

In [179]:
# In the full dataset (before splitting str and numeric columns), there is a column called 'id'
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [180]:
# They're identical. However "loan_data_str[:,5]" is str
print(loan_data_num[:,0])              
print(loan_data_str[:,5])

[48010226. 57693261. 59432726. ... 50415990. 46154151. 66055249.]
['48010226' '57693261' '59432726' ... '50415990' '46154151' '66055249']


In [181]:
# SO CONVERT TO SAME DTYPE
loan_data_num[:,0].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

In [182]:
loan_data_num[:,5].astype(dtype = np.int32)

array([9452, 4679, 1969, ..., 2185, 3199,  301], dtype=int32)

In [183]:
# To verify
np.array_equal(loan_data_num[:,0].astype(dtype = np.int32), loan_data_num[:,5].astype(dtype = np.int32))

# In the video, This is True
# In that case, the URL column doesn't hold any additional info we can't already extract from the ID column
# So in the next cell:

False

In [184]:
# Get rid of the "URL" column
loan_data_str = np.delete(loan_data_str, 5, axis = 1)

# Also from the header
header_str = np.delete(header_str, 5)

In [185]:
# To verify
loan_data_str[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [186]:
# URL is deleted
header_str

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [187]:
# Verify if the IDs remain intact
loan_data_num[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

In [188]:
# "id" is still the first column so == GOOD TO GO
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

##### State Address

In [189]:
# Rename 'addr_state' --> "state_address"
header_str[5] = "state_address"

In [190]:
# Check the unique values
np.unique(loan_data_str[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [191]:
# There are exactly 50 states in the USA so GOOD TO GO
np.unique(loan_data_str[:,5]).size

# But there's an empty space ('') 
# The empty space is IOWA state (IA) which was purposefully left as a baseline "benchmark"

# When doing research or analysis on a variable with many categories,
# it is normal to pick one as a "benchmark" and include dummy variables for the rest

50

In [192]:
np.unique(loan_data_str[:,5], return_counts = True)

(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27]))

In [193]:
# Combine .unique() and .argsort() functions to display the states according to the number of applications in DESCENDING ORDER
# Use tuple assignment
states_names, states_count = np.unique(loan_data_str[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)                                    # Remember "-" sign because we're sorting in DESCENDING ORDER
states_names[states_count_sorted], states_count[states_count_sorted]               # Plug-in [states_count_sorted] as the index

# Most of the data come from the welthiest and most populated states ['CA', 'NY', 'TX', 'FL', '']
# Hence, there are more applications with missing or unreported addresses than there are for 45 of the other states
# Therefore, we have very little data for too many states to examine each one individually
# With that in mind, if we assign a unique value to each state, this will allow OUTLIERS to have a big influence on the coeffs for less represented states
# To elaborate, the more categories a variable has, the fewer data will be available for each one
# As a result, the states with fewer applications will be more vulnerable to have their coeffs affected by OUTLIERS

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10]))

In [194]:
# To solve this problem, we have to group the states according to a certain common characteristic
# A good idea is their GEOGRAPICAL LOCATION

# But first, take care of missing values 
# This way, accounts that cannot be attributed to a specific state won't be included in any region, to avoid biasing with variable coeffs
loan_data_str[:,5] = np.where(loan_data_str[:,5] == '', 
                              0, 
                              loan_data_str[:,5])

In [195]:
# Then:
states_west = np.array(['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM', 'HI', 'AK'])
states_south = np.array(['TX', 'OK', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'FL', 'GA', 'SC', 'NC', 'VA', 'WV', 'MD', 'DE', 'DC'])
states_midwest = np.array(['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH'])
states_east = np.array(['PA', 'NY', 'NJ', 'CT', 'MA', 'VT', 'NH', 'ME', 'RI'])

# Census Regions and Divisions of the United States
# https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [196]:
# Use np.isin() to check whether the values in the column are part of "states_west" and assign 1 if so
# And so on
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_west), 1, loan_data_str[:,5])
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_south), 2, loan_data_str[:,5])
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_midwest), 3, loan_data_str[:,5])
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_east), 4, loan_data_str[:,5])

In [197]:
# Verify 
np.unique(loan_data_str[:,5])

# Where 0 == accounts that cannot be attributed to a specific state

array(['0', '1', '2', '3', '4'], dtype='<U69')

##### Converting Strings

In [198]:
# Converting to Numbers
loan_data_str

# It's full of numbers saved as text (str)

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [199]:
# So convert:
loan_data_str = loan_data_str.astype(np.int)

# We can specify 8 or 16 or 32 
# IF we don't, the function will auto-assign the smallest datatype which succesfully stores all these numbers

loan_data_str

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

##### Checkpoint for text data

In [200]:
# Now create a "checkpoint" to all the changes made
checkpoint_strings = checkpoint("Checkpoint-Strings", header_str, loan_data_str)

In [201]:
checkpoint_strings['header']

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [202]:
checkpoint_strings['data']

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

In [203]:
# To verify if identical
np.array_equal(checkpoint_strings['data'], loan_data_str)

True

### **Manipulating Numeric Data**

In [204]:
loan_data_num

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [205]:
np.isnan(loan_data_num).sum()

# But remember, we filled the missing values with "temporary_fill"

0

##### Substitute Filler Values

In [206]:
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [207]:
temporary_fill

68616520.0

In [208]:
# 'id' column
# We must check whether any of the elements of this column equal the "temporary_fill"
np.isin(loan_data_num[:,0], temporary_fill).sum()

0

In [209]:
# The only column where the min value can be considered worst is the 'funded_amnt'
# Hence, we must switch all "temporary_fill" in that column to min
# While the "rest" to max
# Now we can use the "temporary_stats"
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

In [210]:
# We're only interested in the NUMERIC COLUMNS
temporary_stats[:, columns_numeric]
# min
# mean
# max

# All the columns can contain min and max, EXCEPT 'funded_amnt' as mentioned above
# It should be filled with only the min (worst)

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

In [211]:
# Funded Amount 'funded_amnt' fillers --> min
loan_data_num[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [212]:
loan_data_num[:,2] = np.where(loan_data_num[:,2] == temporary_fill,    # If value is equal to the filler "temporary_fill"
                              temporary_stats[0, columns_numeric[2]],  # assign 0: is the index of min in "temporary_stats", and 
                                                                       # "columns_numeric[2]" == 'funded_amnt' column
                              loan_data_num[:,2])                      # Otherwise remain unchanged
loan_data_num[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [213]:
# Filling out the remaining numeric columns
# 'loan_amnt', 'int_rate', 'installment', 'total_pymnt' fillers --> max
for i in [1,3,4,5]:
  loan_data_num[:,i] = np.where(loan_data_num[:,i] == temporary_fill,    # If value is equal to the filler "temporary_fill"
                                temporary_stats[2, columns_numeric[i]],  # assign 2: is the index of max in "temporary_stats"
                                loan_data_num[:,i])                      # Otherwise remain unchanged

In [214]:
# Now display the dataset
loan_data_num

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

##### Currency Change – The Exchange Rate

In [215]:
# The currency in the dataset is USD, but since the company is based in Europe, convert USD --> EUR
# Import availabe dataset which contains the avg conversion of 2015: USD --> EUR
EURO_USD = np.genfromtxt("/content/drive/MyDrive/Colab Notebooks/365 Data Science/Programming/Numpy/Files for the Practical Example/EUR-USD.csv",
                         delimiter = ',',
                         autostrip = True,
                         skip_header = 1,        # Since we're only interested in the 4th column, dtype can be default and skip the header
                         usecols = 3)           # The 4th column
                         #dtype = np.str)       # To make the headers visible
EURO_USD

# Headers description:
# ['Open', 'High', 'Low', 'Close', 'Volume']
# Open --> Exhange rate at the "beginning" of the trading day
# High --> The daily highest
# Low --> The daily lowest 
# Close --> Exhange rate at the "end" of the trading day (Adjusted Closing Price) 
# Volume --> Number of trades that happened during the given trading day 

# In analysis, we often prefer the "Adjusted Closing Price" so the 4th column

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [216]:
# The issue dates (months) in numerical form
# 0 represents all accounts where the date wasn't provided
# 1-12 Jan to Dec
loan_data_str[:,0]

array([ 5,  0,  9, ...,  6,  4, 12])

In [217]:
# For 1-12 Jan to Dec
exchange_rate = loan_data_str[:,0]

for i in range(1,13):                           # 1 iteration for every month of the year, because the upper limit (13) is not included
  exchange_rate = np.where(exchange_rate == i,  # The function will substitute 1 for January
                           EURO_USD[i-1],       # With the avg exchange rate over that period --> Then Feb, and so on...
                           exchange_rate)
  
# For all accounts where the date wasn't provided (0)
exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EURO_USD),     # The substitute value will be the annual == mean of all exchange rates for 2015
                         exchange_rate)         # If condition is not met, remain unchanged

# Display the new variable
exchange_rate

# Now it contains the assoc. exchange rate for every element of the dataset

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [218]:
# Add the variable "exchange_rate" to the dataset

# Check if has the same shape with "loan_data_num"
print(exchange_rate.shape)
print(loan_data_num.shape)

# exchange_rate is 1D
# loan_data_num is 2D
# Therefore we need to reshape "exchange_rate"

(10000,)
(10000, 6)


In [219]:
# Reshape to match the shape 
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [220]:
# Now, we can "horizontally" add the new array "exchange_rate" to the end of the dataset
loan_data_num = np.hstack((loan_data_num, exchange_rate))
loan_data_num

# Notice the last column == exchange rate

array([[48010226.  ,    35000.  ,    35000.  , ...,     1184.86,     9452.96,        1.1 ],
       [57693261.  ,    30000.  ,    30000.  , ...,      938.57,     4679.7 ,        1.11],
       [59432726.  ,    15000.  ,    15000.  , ...,      494.86,     1969.83,        1.13],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1372.97,     2185.64,        1.12],
       [46154151.  ,    35000.  ,    10000.  , ...,      354.3 ,     3199.4 ,        1.11],
       [66055249.  ,    10000.  ,    10000.  , ...,      309.97,      301.9 ,        1.09]])

In [221]:
# Since we added a column, include it also into the header
header_num = np.concatenate((header_num, np.array(['exchange rate'])))
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange rate'],
      dtype='<U19')

##### Currency Change - From USD to EUR

In [222]:
# 4 columns has values in USD
# 'loan_amnt', 'funded_amnt', 'installment', 'total_pymnt'
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange rate'],
      dtype='<U19')

In [223]:
# Set the columns of interest
columns_dollar = np.array([1,2,4,5])

In [224]:
# Display 
loan_data_num[:,[columns_dollar]]

array([[[35000.  , 35000.  ,  1184.86,  9452.96]],

       [[30000.  , 30000.  ,   938.57,  4679.7 ]],

       [[15000.  , 15000.  ,   494.86,  1969.83]],

       ...,

       [[10000.  , 10000.  ,  1372.97,  2185.64]],

       [[35000.  , 10000.  ,   354.3 ,  3199.4 ]],

       [[10000.  , 10000.  ,   309.97,   301.9 ]]])

In [225]:
# Exchange Rate
loan_data_num[:,6]

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [226]:
# Iterate through "columns_dollar", for every "columns", divide it by "exchange_rate"
# Then add/stack the result to the dataset "horizontally"
for i in columns_dollar:
  loan_data_num = np.hstack((loan_data_num, np.reshape(loan_data_num[:,i] / loan_data_num[:,6], (10000,1))))

#Reshape because the dataset is 2D while the exchange_rate is 1D

In [227]:
# The dataset now is 10000x11 
print(loan_data_num)
print("")
print(loan_data_num.shape)

[[48010226.      35000.      35000.   ...    31933.3      1081.04     8624.69]
 [57693261.      30000.      30000.   ...    27132.46      848.86     4232.39]
 [59432726.      15000.      15000.   ...    13326.3       439.64     1750.04]
 ...
 [50415990.      10000.      10000.   ...     8910.3      1223.36     1947.47]
 [46154151.      35000.      10000.   ...     8997.4       318.78     2878.63]
 [66055249.      10000.      10000.   ...     9145.8       283.49      276.11]]

(10000, 11)


In [228]:
# Update the header
# This comprehension goes through "columns_dollar" and patches '_EUR' to each "column_name"
header_additional = np.array([column_name + '_EUR' for column_name in header_num[columns_dollar]])
header_additional

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [229]:
# Add the created array to the header
header_num = np.concatenate((header_num, header_additional))
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [230]:
# Rename the original USD columns aswell for more clarity
header_num[columns_dollar] = np.array([column_name + '_USD' for column_name in header_num[columns_dollar]])
header_num

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [231]:
# Rearrange the columns so that each EUR column follows its corresponding USD column
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [232]:
# set "columns_index_order" as index
header_num = header_num[columns_index_order]
header_num

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange rate'],
      dtype='<U19')

In [233]:
# Set the order "columns_index_order" aswell to the dataset
loan_data_num = loan_data_num[:,columns_index_order]
loan_data_num

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

##### Interest Rate

In [234]:
header_num

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange rate'],
      dtype='<U19')

In [235]:
# The values are in percentage form
# The CONVENTION usually dictates using values between 0 and 1
# Because it makes certain calculations, like estimating interest payments simple
loan_data_num[:,5]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [237]:
# Transform the "int_rate" column and overwrite
loan_data_num[:,5] = loan_data_num[:,5] / 100
loan_data_num[:,5]

##### Checkpoint for Numerical Values

In [239]:
checkpoint_numeric = checkpoint("checkpoint-numeric", header_num, loan_data_num)

In [240]:
 checkpoint_numeric['header'], checkpoint_numeric['data']

(array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
        'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange rate'],
       dtype='<U19'),
 array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
        [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
        [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
        ...,
        [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
        [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
        [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]]))

### **Completing the Dataset**
- **Summary:**
  - Splitted the org array into 2
  - Cleaned and preprocessed each part separately
  - Stitch them back together

##### Final check

In [243]:
# The 2 arrays must have compatible shapes

#To check
#print(loan_data_num.shape)
#print(loan_data_str.shape)

# Or use the SAFER: (Incase we accidentally altered some contents of the string dataset after storing)
print(checkpoint_strings['data'].shape)
print(checkpoint_numeric['data'].shape)

# Both has the same "ROWS" so we we can stack them "side-by-side" (Horizontally)

(10000, 6)
(10000, 11)


In [245]:
print(np.hstack((checkpoint_numeric['data'], checkpoint_strings['data'])))
print(np.hstack((checkpoint_numeric['data'], checkpoint_strings['data'])).shape)

[[48010226.      35000.      31933.3  ...       13.          1.          1.  ]
 [57693261.      30000.      27132.46 ...        5.          1.          4.  ]
 [59432726.      15000.      13326.3  ...       10.          1.          4.  ]
 ...
 [50415990.      10000.       8910.3  ...        5.          1.          1.  ]
 [46154151.      35000.      31490.9  ...       17.          1.          3.  ]
 [66055249.      10000.       9145.8  ...        4.          0.          3.  ]]
(10000, 17)


In [246]:
# Set it to a variable
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))
loan_data

array([[48010226.  ,    35000.  ,    31933.3 , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27132.46, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

In [247]:
# Check for missing values
np.isnan(loan_data).sum()

0

In [248]:
# Set the header to the combined dataset
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))
header_full

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange rate',
       'issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

##### Sorting the NEW DATASET

In [252]:
# Rearrange the entire dataset according to the values in the first column 'id'
loan_data = loan_data[np.argsort(loan_data[:,0])]
loan_data

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [253]:
# To verify (should start at 0 and so on)
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999])

##### Storing the NEW DATASET

In [254]:
# After sorting the array, stack the header ontop "np.vstack()"
# NOTE: The stack requires a unified datatype accross all the rows and columns
# The function selects the smallest datatype which can hold any of the elements within the array
loan_data = np.vstack((header_full, loan_data))
loan_data

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'state_address'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

In [255]:
# Save the dataset
np.savetxt("/content/drive/MyDrive/Colab Notebooks/365 Data Science/Programming/Numpy/Files for the Practical Example/loan-data-preprocessed.csv", 
           loan_data,                        # The loan_data variable
           fmt = "%s",                       # String format
           delimiter = ",") 

# Check Directory --> Checkfile with notepad                 